In [1]:
#pip install importnb

In [25]:
# This notebook executes the training routines of the agent(s) 
import random
import time
from importnb import Notebook
with Notebook():
    import Board
    import Agent
from abc import ABC, abstractmethod

In [27]:
#### Parent Class ####

In [29]:
class Game(ABC):
    def __init__(self, board, agent1):
        """
        clase abstracta Game
        """
        self.board = board
        self.agent1 = agent1

    def reset_game(self):
        self.board.reset()

    @abstractmethod
    def play(self):
        pass # To be implemented by the child classes

In [31]:
#### Child Classes ####

In [33]:
class AgentVsAgent(Game):
    def __init__(self, board, agent1, agent2, epochs = 1):
        super().__init__(board, agent1)
        self.epochs = epochs
        self.agent2 = agent2
        if agent1.chip == agent2.chip:
            raise ValueError("Los agentes deben tener fichas diferentes")

    def play(self, save_learning = True):
        """Implementa los juegos entre los dos agentes."""
        
        for epoch in range(self.epochs):
            start = time.time()
            self.reset_game()
            agent_1_turn = random.choice([True, False]) # randomize who gets first turn

            # An EPOCH
            while True :                
                
                if agent_1_turn:# Turno Agente 1
                    self.agent1.play_turn(episode = epoch)  
                        
                else: # Turno Agente 2
                    self.agent2.play_turn(episode = epoch)

                if self.board.isBoardFull() or self.board.verify_winner(self.agent1.chip) or self.board.verify_winner(self.agent2.chip):
                    break
         
                agent_1_turn = not agent_1_turn
            # END OF AN EPOCH
            
            end = time.time()
            #if epoch%epoch == 0:
            print(f"Epoch # {epoch} took {(end - start):.3f} s")
        if save_learning:
            self.agent1.export_Qtable('Q_Table_A1.pkl')
            self.agent1.export_Qtable_JSON('Q_Table_A1.json')
            self.agent2.export_Qtable('Q_Table_A2.pkl')
            self.agent2.export_Qtable_JSON('Q_Table_A2.json')

In [35]:
class PlayerVsAgent(Game):
    def __init__(self, board, agent):
        super().__init__(board, agent)
        self.agent = agent
        #self.player_chip = ?
        if self.agent.chip == "X":
            self.player_chip = "O"
        else:
            self.player_chip = "X"

    def play(self):
        """Implementa un juego entre un humano y un agente."""
                
        play_again = True
        while play_again:
            
            self.reset_game()
            player_turn = random.choice([True, False]) # randomize who gets first turn
            if player_turn:
                print("You start this time")
            else:
                print("I will start this time")
            
            while not (self.board.isBoardFull() or self.board.verify_winner(self.player_chip) or self.board.verify_winner(self.agent.chip)):         

                
                if player_turn:
                    # print the current state of the board
                    self.board.print_board()
                    print("It´s your turn!")

                    # Ask for the next move and place the chip
                    while True:
                        try:
                            user_input = input("What is your move? ")
                            new_col_number = int(user_input)  # Intentar convertir a entero
                            
                            # Verificar que el número esté dentro del rango permitido
                            if not (1 <= new_col_number <= 7):
                                raise ValueError("The column must be a number between 1 and 7.")
                            
                            # Intentar colocar la ficha en el tablero
                            if not self.board.place_chip(new_col_number, self.player_chip):
                                raise ValueError("Invalid move. The column might be full or invalid. Try again.")
                            
                            # Si el movimiento es válido, romper el bucle
                            break
                            
                        except ValueError as e:
                            print(e)

                # FIN DEL TURNO DEL Jugador
                    
                else: # Agents Turn

                    # Verify if the player won on the previous move
                    if self.board.verify_winner(self.player_chip):
                        print("YOU WIN!")
                        # comunicarle al agente que perdió -> asignar la recompensa 
                        break
                    elif self.board.isBoardFull():
                        print("ITS A TIE")
                        # comunicarle al agente que empato -> asignar la recompensa 
                        break
                    
                    print("\nIt´s My turn!")
                    self.agent.play_turn()

                    if self.board.verify_winner(self.agent.chip):
                        print("YOU LOSE!")
                        break
                    # FIN DEL TURNO DEL AGENTE

                player_turn = not player_turn

            # Juego terminó, preguntar si se quiere volver a jugar
            usr_input = ""
            while usr_input.lower() not in ["y", "n", "yes", "no"]:
                usr_input = input("Do you want to play again? (y/n): ").strip()

            if usr_input.lower() in ["y", "yes"]:
                play_again = True  # No es necesario redefinir, pero preferi dejarlo explícito
            elif usr_input.lower() in ["n", "no"]:
                play_again = False

        print("Thanks for playing, Bye!")

**Entrenamiento Agente VS Agente**

In [37]:
tablero = Board.Connect4Board()

agente = Agent.Agent(tablero, "O", epsilon = 0.95, alpha = 0.8, gamma = 1)
otroAgente = Agent.Agent(tablero, "X", epsilon = 0.95, alpha = 0.2, gamma = 1)
juego = AgentVsAgent(tablero, agente, otroAgente, epochs = 300)

juego.play()

Epoch # 0 took 0.023 s
Epoch # 1 took 0.047 s
Epoch # 2 took 0.053 s
Epoch # 3 took 0.053 s
Epoch # 4 took 0.054 s
Epoch # 5 took 0.020 s
Epoch # 6 took 0.075 s
Epoch # 7 took 0.033 s
Epoch # 8 took 0.040 s
Epoch # 9 took 0.018 s
Epoch # 10 took 0.033 s
Epoch # 11 took 0.077 s
Epoch # 12 took 0.014 s
Epoch # 13 took 0.055 s
Epoch # 14 took 0.048 s
Epoch # 15 took 0.054 s
Epoch # 16 took 0.047 s
Epoch # 17 took 0.040 s
Epoch # 18 took 0.050 s
Epoch # 19 took 0.038 s
Epoch # 20 took 0.039 s
Epoch # 21 took 0.057 s
Epoch # 22 took 0.053 s
Epoch # 23 took 0.040 s
Epoch # 24 took 0.040 s
Epoch # 25 took 0.022 s
Epoch # 26 took 0.059 s
Epoch # 27 took 0.069 s
Epoch # 28 took 0.030 s
Epoch # 29 took 0.045 s
Epoch # 30 took 0.088 s
Epoch # 31 took 0.019 s
Epoch # 32 took 0.044 s
Epoch # 33 took 0.047 s
Epoch # 34 took 0.066 s
Epoch # 35 took 0.049 s
Epoch # 36 took 0.043 s
Epoch # 37 took 0.021 s
Epoch # 38 took 0.036 s
Epoch # 39 took 0.041 s
Epoch # 40 took 0.018 s
Epoch # 41 took 0.034 s
Ep

**Prueba contra un agente aleatorio**

In [15]:
agenteCarga = Agent.Agent(tablero, "O", filename = 'Q_Table_A1.pkl')
#juego = PlayerVsAgent(tablero,agenteCarga)
#tabla = agenteCarga.q_table


Q_Table_A1.pkl
Q-table load successfull new
{(((0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)), 6): 3.125, (((0, 0), (0, 0), (0, 0), (0, 0), (1, 1), (0, 0), (1, 1)), 3): 0.0, (((0, 0), (0, 0), (1, 1), (1, 1), (1, 1), (0, 0), (1, 1)), 7): 12.5, (((1, 1), (0, 0), (1, 1), (1, 1), (2, 2), (1, 1), (1, 1)), 5): 63.125, (((1, 1), (0, 0), (1, 1), (1, 1), (2, 2), (1, 1), (0, 0)), 7): 0.0, (((0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)), 4): 1.5625, (((0, 0), (0, 0), (1, 1), (1, 1), (1, 1), (0, 0), (0, 0)), 7): 0.0, (((1, 1), (0, 0), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)), 4): 28.75, (((1, 1), (0, 0), (1, 1), (1, 1), (2, 2), (1, 1), (1, 1)), 2): 16.25, (((1, 1), (0, 0), (1, 1), (1, 1), (2, 2), (1, 1), (1, 1)), 4): 12.5, (((1, 1), (0, 0), (1, 1), (1, 1), (3, 3), (1, 1), (0, 0)), 3): 12.5, (((3, 3), (2, 2), (0, 0), (1, 1), (3, 3), (1, 1), (0, 0)), 7): 0.0, (((2, 2), (2, 2), (0, 0), (1, 1), (0, 0), (1, 1), (1, 1)), 6): 12.5, (((2, 2), (2, 2), (0, 0), (1, 1), (0, 0), (1, 1), (1, 1